In [7]:
import os
import time
import shutil

import torch
import torch.nn as nn
from torch.backends import cudnn

import torchvision
import torchvision.transforms as transforms

from models import *

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"


full_model_path = f"result/VGG16_Full/model_best.pth.tar"
batch_size = 64
lr = 3e-3
epochs = 100
prune_schedule = {0: 5/9,
                  10: 1/9,
                  20: 1/9}
print_freq = 100
sparsity = sum(prune_schedule.values())
model_name = f"VGG16_new_os_iter_prune_{sparsity:.2f}"
fdir = 'result/' + model_name


model = VGG16()
checkpoint = torch.load(full_model_path)
model.load_state_dict(checkpoint['state_dict'])
model.cuda()
device = torch.device("cuda")

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)


normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])
train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

def train_epoch(trainloader, model, criterion, optimizer, epoch):

    model.train()
    
    num_correct = 0
    num_processed = 0
    end = time.time()
    for i, (input, target) in enumerate(trainloader, start=1):
        
        input, target = input.cuda(), target.cuda()

        output:torch.Tensor = model(input)
        loss = criterion(output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        num_correct += output.argmax(dim=1).eq(target).sum().item()
        num_processed += output.size(dim=0)
        
        if i % print_freq == 0:
            print(f'Epoch: {epoch} ({i}/{len(trainloader)})\t'
                  f'Time Elapsed: {(time.time() - end):.1f}s\t'
                  f'Epoch Accuracy: {num_correct/num_processed:.1%}')
        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))

def train_model(model, fdir, criterion, optimizer, epochs, prune_schedule:dict=None): 
    
    os.makedirs(fdir, exist_ok=True)

    best_prec = 0

    model.cuda()
    criterion = criterion.cuda()
    cudnn.benchmark = True

    for epoch in range(0, epochs):

        if prune_schedule is not None and epoch in prune_schedule: 
            os_prune_vgg16(model, prune_schedule[epoch])

        train_epoch(trainloader, model, criterion, optimizer, epoch)
        prec = val_model(model)
        
        if prune_schedule is None or (prune_schedule is not None and epoch >= list(prune_schedule.keys())[-1]): 
            is_best = prec > best_prec
            best_prec = max(prec,best_prec)
            print('best acc: {:1f}'.format(best_prec))
            
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'best_prec': best_prec,
                'optimizer': optimizer.state_dict(),
            }, is_best, fdir)

def val_model(model): 

    model.cuda()
    model.eval()

    correct = 0
    with torch.no_grad():
        for data, target in testloader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            correct += output.argmax(dim=1).eq(target).sum().item()
            
    print(f'Test Set Accuracy: {correct}/{len(testloader.dataset)} ({correct/len(testloader.dataset):.1%})')

/tmp/ipykernel_109046/2199682218.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(full_model_path)


Files already downloaded and verified
Files already downloaded and verified


In [8]:
train_model(model, fdir, criterion, optimizer, epochs, prune_schedule)

Pruning 36 ic-slices out of 64 ic-slices (56.2% pruned)
Pruning 36 ic-slices out of 64 ic-slices (56.2% pruned)
Pruning 71 ic-slices out of 128 ic-slices (55.5% pruned)
Pruning 71 ic-slices out of 128 ic-slices (55.5% pruned)
Pruning 142 ic-slices out of 256 ic-slices (55.5% pruned)
Pruning 142 ic-slices out of 256 ic-slices (55.5% pruned)
Pruning 142 ic-slices out of 256 ic-slices (55.5% pruned)
Pruning 284 ic-slices out of 512 ic-slices (55.5% pruned)
Pruning 284 ic-slices out of 512 ic-slices (55.5% pruned)
Pruning 284 ic-slices out of 512 ic-slices (55.5% pruned)
Pruning 284 ic-slices out of 512 ic-slices (55.5% pruned)
Pruning 284 ic-slices out of 512 ic-slices (55.5% pruned)
Epoch: [0][100/782]	Time Elapsed: 5.0s	Running Average Accuracy: 53.6%
Epoch: [0][200/782]	Time Elapsed: 9.9s	Running Average Accuracy: 62.6%
Epoch: [0][300/782]	Time Elapsed: 14.8s	Running Average Accuracy: 67.3%
Epoch: [0][400/782]	Time Elapsed: 19.8s	Running Average Accuracy: 70.7%
Epoch: [0][500/782]	Time

KeyboardInterrupt: 

In [3]:
model = VGG16()
os_prune_vgg16(model, sparsity)

PATH = f"{fdir}/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
model.cuda()

val_model(model)

print_sparsity(model)

Pruning 50 ic-slices out of 64 ic-slices (78.1% pruned)
Pruning 50 ic-slices out of 64 ic-slices (78.1% pruned)
Pruning 100 ic-slices out of 128 ic-slices (78.1% pruned)
Pruning 100 ic-slices out of 128 ic-slices (78.1% pruned)
Pruning 200 ic-slices out of 256 ic-slices (78.1% pruned)
Pruning 200 ic-slices out of 256 ic-slices (78.1% pruned)
Pruning 200 ic-slices out of 256 ic-slices (78.1% pruned)
Pruning 399 ic-slices out of 512 ic-slices (77.9% pruned)
Pruning 399 ic-slices out of 512 ic-slices (77.9% pruned)
Pruning 399 ic-slices out of 512 ic-slices (77.9% pruned)
Pruning 399 ic-slices out of 512 ic-slices (77.9% pruned)
Pruning 399 ic-slices out of 512 ic-slices (77.9% pruned)


/tmp/ipykernel_39922/3582425983.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH)



Test set: Accuracy: 8668/10000 (87%)

layer 3 sparsity: 0.781
layer 7 sparsity: 0.781
layer 10 sparsity: 0.773
layer 14 sparsity: 0.773
layer 17 sparsity: 0.773
layer 20 sparsity: 0.773
layer 24 sparsity: 0.773
layer 27 sparsity: 0.777
layer 30 sparsity: 0.777
layer 34 sparsity: 0.777
layer 37 sparsity: 0.777
layer 40 sparsity: 0.777
